In [106]:
import scipy as sp
import scipy.stats as sps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns
import jenkspy as jp
import math
from sklearn.manifold import MDS
from itertools import pairwise, combinations,product

In [108]:
def precision(answer, targets):
    retrieved_relevant = set(answer).intersection(set(targets))
    return len(retrieved_relevant) / len(answer)


def recall(answer, targets):
    relevant = set(targets)
    retrieved_relevant = set(answer).intersection(relevant)
    return len(retrieved_relevant) / len(relevant)


def f1score(answer, targets):
    return 2 / (1 / recall(answer, targets) + 1 / precision(answer, targets))


recall([1, 2, 3], [2, 1, 4, 5, 6, 7])

0.3333333333333333

In [145]:
df = pd.read_csv('./pilot.csv')
pd.set_option('display.max_rows', 500)
df[df.columns[:15]]

,id,submitdate,lastpage,startlanguage,seed,startdate,datestamp,consent,abort,screen,screeningsinfo,age,vision,color,dyslexia
0,18,NaN,24,en,514157120,2024-04-02 08:25:43,2024-04-02 08:36:16,Yes,NaN,1680;912;2,NaN,33,Yes,No,No
1,19,2024-04-02 12:06:47,57,en,342518396,2024-04-02 11:38:49,2024-04-02 12:06:47,Yes,NaN,2560;1279;1,NaN,36,Yes,No,No
2,20,2024-04-02 13:49:08,57,en,311441978,2024-04-02 13:18:52,2024-04-02 13:49:08,Yes,NaN,1920;1080;1,NaN,30,Yes,No,No
3,21,2024-04-02 13:53:44,57,en,855284146,2024-04-02 13:28:54,2024-04-02 13:53:44,Yes,NaN,2560;1325;1,NaN,22,Yes,No,No
4,22,2024-04-04 12:40:56,57,en,517357640,2024-04-04 12:16:54,2024-04-04 12:40:56,Yes,NaN,1920;959;2,NaN,35,Yes,No,No
5,23,2024-04-05 09:49:22,57,en,449140046,2024-04-05 09:03:55,2024-04-05 09:49:22,Yes,NaN,1920;989;1,NaN,24,Yes,No,No
6,24,2024-04-07 20:40:29,57,en,2100952373,2024-04-07 19:57:22,2024-04-07 20:40:29,Yes,NaN,2560;1287;1,NaN,29,Yes,No,No


In [148]:
# exclude data

# only complete responses (they have a submit date)
df = df[pd.notna(df["submitdate"])]
# only consent - no technical reasons why they would still be here but you never know
df = df[df["consent"] == "Yes"]
# no dyslexia
df = df[df["dyslexia"] == "No"]
# no vision or color impairment
df = df[(df["vision"] == "Yes") & (df["color"] == "No")]
# age: 18–120
df = df[(df["age"] >= 18) & (df["age"] <= 120)]

# wrong answers to control questions
# control questions have glyph 40 as reference
# the target cluster is glyphs 36,37,38,39,40,41
# so correct answers are a permutation of 36,37,38,39,41

df['control4']
# TODO exclude more based on vision, dyslexia, age, control questions...

#df

1    41,39,38,36,37;4151
2    37,39,38,36,41;7524
3    37,41,39,38,36;3581
4    41,39,38,36,37;7058
5    36,38,39,41,37;5150
6    36,38,39,41,37;7543
Name: control4, dtype: object

In [109]:
conditions = list(product(['D1','D2','D3', 'D4'], ['Ph','Po'], ['W1','W5','W0']))
experiment_questions = list(map(lambda x:  f"exp{''.join(x)}",conditions))
experiment_questions

['expD1PhW1',
 'expD1PhW5',
 'expD1PhW0',
 'expD1PoW1',
 'expD1PoW5',
 'expD1PoW0',
 'expD2PhW1',
 'expD2PhW5',
 'expD2PhW0',
 'expD2PoW1',
 'expD2PoW5',
 'expD2PoW0',
 'expD3PhW1',
 'expD3PhW5',
 'expD3PhW0',
 'expD3PoW1',
 'expD3PoW5',
 'expD3PoW0',
 'expD4PhW1',
 'expD4PhW5',
 'expD4PhW0',
 'expD4PoW1',
 'expD4PoW5',
 'expD4PoW0']

In [111]:
exp = df[experiment_questions]

d1 = pd.read_csv('./dataset/ds_dataset1.csv')
d2 = pd.read_csv('./dataset/ds_dataset2.csv')
d3 = pd.read_csv('./dataset/ds_dataset3.csv')
d4 = pd.read_csv('./dataset/ds_dataset4.csv')

target_d1 = [i for i in d1[d1['cluster']==1].index.tolist() if i != 40 ]
target_d2 = d2[d2['S4']==1].index.tolist()
target_d3 = d3[(d3['S0']==1) & (d3['S2']==1)].index.tolist()
target_d4 = [ i for i in d4[(d4['cluster']==1) & ((d4['S1']==1)|(d4['S2']==1))].index.tolist() if i != 37]

def get_target(q):
    match q[4]:
        case '1': return target_d1
        case '2': return target_d2
        case '3': return target_d3
        case '4': return target_d4
    raise BaseException(q[4])


for q in experiment_questions:
    exp[f"{q}_t"] = pd.NA
    exp[f"{q}_a"] = pd.NA
    for i in exp.index:
        timed_response = exp.loc[i,q]
        exp.loc[i,f"{q}_t"] = int(timed_response.split(';')[1])
        response = [int(i) for i in timed_response.split(';')[0].split(',')]
        exp.loc[i,f"{q}_a"] = f1score(response, get_target(q))

exp = exp.drop(columns=experiment_questions)
exp


/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/3400096289.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp[f"{q}_t"] = pd.NA
/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/3400096289.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp[f"{q}_a"] = pd.NA
/var/folders/d9/8l4xsjcd5_59v2f35x1y8dgm0000gn/T/ipykernel_37222/3400096289.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,expD1PhW1_t,expD1PhW1_a,expD1PhW5_t,expD1PhW5_a,expD1PhW0_t,expD1PhW0_a,expD1PoW1_t,expD1PoW1_a,expD1PoW5_t,expD1PoW5_a,...,expD4PhW5_t,expD4PhW5_a,expD4PhW0_t,expD4PhW0_a,expD4PoW1_t,expD4PoW1_a,expD4PoW5_t,expD4PoW5_a,expD4PoW0_t,expD4PoW0_a
1,27741,0.888889,11330,0.571429,7755,1.0,24146,0.888889,22837,1.0,...,40011,0.857143,20301,1.0,32342,1.0,22987,0.666667,38214,1.0
2,34320,1.0,17905,0.888889,11190,1.0,20482,1.0,20321,1.0,...,31251,1.0,33897,0.857143,26088,0.857143,21923,0.4,56907,1.0
3,17481,0.888889,20062,1.0,7629,1.0,24061,1.0,18012,0.888889,...,46482,1.0,17774,1.0,24000,1.0,24542,1.0,24075,1.0
4,17593,0.888889,27592,0.888889,12257,1.0,29024,0.75,22957,0.571429,...,34456,0.666667,26456,0.857143,30990,1.0,38388,0.857143,28855,0.857143
5,29713,1.0,27578,1.0,9072,1.0,20682,0.571429,31808,0.888889,...,29802,0.857143,33090,1.0,29442,0.666667,28771,1.0,24744,0.857143
6,65946,1.0,55165,0.75,22866,1.0,42291,0.888889,24894,0.75,...,53472,0.857143,24837,1.0,51110,1.0,77298,0.857143,69998,1.0


In [131]:
# convert to long form
import csv

exp["participant"] = exp.index
exp_long = pd.melt(exp, id_vars=["participant"], var_name="stimulus_response")
exp_long["task"] = exp_long.apply(lambda row: int(row["stimulus_response"][4]), axis=1)
exp_long["alpha"] = exp_long.apply(lambda row: int(row["stimulus_response"][8]), axis=1)
exp_long["pipeline"] = exp_long.apply(lambda row: row["stimulus_response"][6], axis=1)
exp_long["metric"] = exp_long.apply(lambda row: row["stimulus_response"][-1], axis=1)
exp_long = exp_long.drop(columns="stimulus_response")
exp_long.to_csv("./survey_long.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)